In [1]:
import numpy as np
import pandas as pd

In [4]:
row_data=pd.read_excel('cloze_task_raw_data.xlsx')

In [5]:
list(row_data.columns)

['shown',
 'answer',
 'word.id',
 'item.id',
 'word.serial.no',
 'subject.id',
 'item.text',
 'accuracy',
 'fl',
 'lang',
 'sex',
 'reading',
 'age',
 'Lemma',
 'POS',
 'Form']

In [6]:
res = row_data[['shown',
 'answer',
 'word.id',
 'item.id',
 'word.serial.no',
 'subject.id',
 'item.text',
 'accuracy',
 'fl',
 'lang',
 'sex',
 'reading',
 'age',
 'Lemma',
 'POS',
 'Form']]
print(res.shape)
res = res[res['shown']!='Введите первое слово']
print(res.shape)
res = res.dropna(subset=['answer'])
print(res.shape)
def clear(x):
    flag=0
    if (len(x)==1) and (x not in 'ксявуоиа'):
        flag+=1
    for i in x:
        if i not in 'ячсмитьбюэждлорпавыфйцукенгшщзхъЯЧСМИТЬБЮФЫВАПРОЛДЖЭЪХЗЩЙЦШУГКНЕ':
             flag+=1
    if flag==0:
        return x
    else:
        return 0
res.answer = res.answer.astype(str)
res.answer = res.answer.apply(clear)
res = res[res['answer']!=0]
print(res.shape)


(64645, 16)
(57836, 16)
(57804, 16)
(57346, 16)


In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def tagging(x):
    return str(morph.parse(x)[0].tag.POS)

res['tag'] = res['answer'].apply(tagging)
res.head()

shown  answer    word.id  item.id  word.serial.no subject.id  \
0     А     вот  промывать      102               2     SKX694   
1     А     где  промывать      102               2     ERY716   
2     А     ему  промывать      102               2     JAD570   
3     А   зачем  промывать      102               2     INH525   
4     А  знаете  промывать      102               2     AOL927   

                                           item.text  accuracy  fl     lang  \
0  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
1  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
2  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
3  А промывать манную крупу перед тем‚ как варить...       0.0   0  Русский   
4  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   

      sex reading   age      Lemma POS            Form   tag  
0  female   min90  25.0  промывать   V  V=инф,несов,пе  PRCL  
1  female   a lot  42.0  промывать   V  V=инф,несов,пе  ADVB  
2    male   min60  42.0  промывать   V  V=инф,несов,пе  NPRO  
3  female   min90  20.0  промывать   V  V=инф,несов,пе  ADVB  
4  female   min60  28.0  промывать   V  V=инф,несов,пе  CONJ

In [8]:
from pymystem3 import Mystem
m = Mystem()

def tag_mystem(x):
    return m.analyze(x)[0]['analysis']

res['tag_stem'] = res['answer'].apply(tag_mystem)
res.head()

shown  answer    word.id  item.id  word.serial.no subject.id  \
0     А     вот  промывать      102               2     SKX694   
1     А     где  промывать      102               2     ERY716   
2     А     ему  промывать      102               2     JAD570   
3     А   зачем  промывать      102               2     INH525   
4     А  знаете  промывать      102               2     AOL927   

                                           item.text  accuracy  fl     lang  \
0  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
1  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
2  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
3  А промывать манную крупу перед тем‚ как варить...       0.0   0  Русский   
4  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   

      sex reading   age      Lemma POS            Form   tag  \
0  female   min90  25.0  промывать   V  V=инф,несов,пе  PRCL   
1  female   a lot  42.0  промывать   V  V=инф,несов,пе  ADVB   
2    male   min60  42.0  промывать   V  V=инф,несов,пе  NPRO   
3  female   min90  20.0  промывать   V  V=инф,несов,пе  ADVB   
4  female   min60  28.0  промывать   V  V=инф,несов,пе  CONJ   

                                            tag_stem  
0  [{'lex': 'вот', 'wt': 0.9998923304, 'gr': 'PAR...  
1         [{'lex': 'где', 'wt': 1, 'gr': 'ADVPRO='}]  
2  [{'lex': 'он', 'wt': 0.9886697737, 'gr': 'SPRO...  
3       [{'lex': 'зачем', 'wt': 1, 'gr': 'ADVPRO='}]  
4  [{'lex': 'знать', 'wt': 1, 'gr': 'V,несов,пе=н...

In [9]:
def tag_my(x):
    if len(x)>=1:
        f = x[0]['gr'].split(',')[0]
        s = ''.join([j for j in f if j.isupper()])
        return s
    else:
        return x

res['tag_stem'] = res['tag_stem'].apply(tag_my)


In [10]:
res.head()

shown  answer    word.id  item.id  word.serial.no subject.id  \
0     А     вот  промывать      102               2     SKX694   
1     А     где  промывать      102               2     ERY716   
2     А     ему  промывать      102               2     JAD570   
3     А   зачем  промывать      102               2     INH525   
4     А  знаете  промывать      102               2     AOL927   

                                           item.text  accuracy  fl     lang  \
0  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
1  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
2  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   
3  А промывать манную крупу перед тем‚ как варить...       0.0   0  Русский   
4  А промывать манную крупу перед тем‚ как варить...       0.0   0  русский   

      sex reading   age      Lemma POS            Form   tag tag_stem  
0  female   min90  25.0  промывать   V  V=инф,несов,пе  PRCL     PART  
1  female   a lot  42.0  промывать   V  V=инф,несов,пе  ADVB   ADVPRO  
2    male   min60  42.0  промывать   V  V=инф,несов,пе  NPRO     SPRO  
3  female   min90  20.0  промывать   V  V=инф,несов,пе  ADVB   ADVPRO  
4  female   min60  28.0  промывать   V  V=инф,несов,пе  CONJ        V

In [11]:
mst_to_pm = {'A':'ADJ', 'ADJF':'ADJ', 'ADJS':'ADJ', 'ADV':'ADVB', 'ADVPRO':'ADVB', 'ANUM':'NUMR', 
             'APRO':'ADJ', 'NUM':'NUMR', 'PART':'PRCL', 'PRTF' : 'VERB', 'PRTS' : 'VERB',
             'PR':'PREP', 'S':'NOUN', 'SPRO':'NPRO', 'V':'VERB', 'COMP' : 'ADJ'}
def replace_tags(x):
    if x in list(mst_to_pm.keys()):
        return mst_to_pm[x]
    else:
        return x
    
        
res['new_tag_stem'] = res['tag_stem'].apply(replace_tags)
res['new_tag_morphy'] = res['tag'].apply(replace_tags)

### Measure of consistancy

In [13]:
res['compare'] = np.where(res['new_tag_stem']==res['new_tag_morphy'], True,False)
print(res['compare'][res['compare']==True].shape[0]/res.shape[0])

0.8184005859170649


In [14]:
res['answer']

0             вот
1             где
2             ему
3           зачем
4          знаете
5            зори
6           когда
7             кто
8              он
10           петя
11            тем
12            тот
13             ты
14            что
15              я
16          глаза
17          горло
18        желудок
19             ли
20       макароны
21          мозги
22          мозги
23          мозги
24        морковь
25            нос
27           рану
28           рану
29           рану
30           рану
31            рис
           ...   
64615         где
64616         где
64617         где
64618        есть
64619         как
64620         как
64621         как
64622      каково
64623         кто
64624        куда
64625          не
64626         что
64627         что
64628         что
64629         что
64630          ли
64631          ли
64632          ли
64633          ли
64634          ли
64635          ли
64636          ли
64637          ли
64638          ли
64639     

In [15]:
res.to_excel('result_tag_new.xls')